# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of skills that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [13]:
#r "nuget: Microsoft.SemanticKernel, 0.13.442.1-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.CoreSkills;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Planning.Sequential;

IKernel kernel = KernelBuilder.Create();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();


if (useAzureOpenAI)
    kernel.Config.AddAzureTextCompletionService( model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletionService( model, apiKey, orgId);


Installed Packages Microsoft.SemanticKernel, 0.13.442.1-preview

### Setting Up the Planner
The planner is located in the Semantic Kernel's CoreSkills and requires Orchestration

In [14]:
// Load native skill into the kernel registry, sharing its functions with prompt templates
 var planner = new SequentialPlanner(kernel);

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk.

In [15]:
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "skills");
kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "SummarizeSkill");
kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "WriterSkill");

Define your ASK. What do you want the Kernel to do?

In [16]:
var ask = "Tomorrow is Valentine's day. I need to come up with a few date ideas and e-mail them to my significant other.";
var originalPlan = await planner.CreatePlanAsync(ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(Utils.ToPlanString(originalPlan));



Original plan:

 Goal: Tomorrow is Valentine's day. I need to come up with a few date ideas and e-mail them to my significant other.

 Steps:
  - WriterSkill.Brainstorm INPUT='Valentine's Day Date Ideas' => DATE_IDEAS
  - WriterSkill.EmailGen INPUT='$DATE_IDEAS' => EMAIL_TEXT
  - Microsoft.SemanticKernel.Planning.Plan. INPUT=''


As you can see in the above plan, the Planner has taken the user's ask and converted it into an XML-based plan detailing how the AI would go about solving this task.

It makes use of the skills that the Kernel has available to it and determines which functions to call in order to fullfill the user's ask.

The output of each step of the plan gets set as `setContextVariable` which makes it available as `input` to the next skill.

Let's also define an inline skill and have it be available to the Planner.
Be sure to give it a function name and skill name.

In [17]:
string skPrompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
""";
var shakespeareFunction = kernel.CreateSemanticFunction(skPrompt, "shakespeare", "ShakespeareSkill", maxTokens: 2000, temperature: 0.2, topP: 0.5);

Let's update our ask using this new skill.

In [18]:
var ask = @"Tomorrow is Valentine's day. I need to come up with a few date ideas.
 She likes Shakespeare so write using his style. E-mail these ideas to my significant other";
var newPlan = await  planner.CreatePlanAsync(ask);
Console.WriteLine("Updated plan:\n");
Console.WriteLine(Utils.ToPlanString(newPlan));

Updated plan:

 Goal: Tomorrow is Valentine's day. I need to come up with a few date ideas.
 She likes Shakespeare so write using his style. E-mail these ideas to my significant other

 Steps:
  - WriterSkill.Brainstorm INPUT='Valentine's day date ideas' => IDEAS
  - ShakespeareSkill.shakespeare INPUT='$IDEAS' => SHAKESPEARE_IDEAS
  - Microsoft.SemanticKernel.Planning.Plan. INPUT=''
  - Microsoft.SemanticKernel.Planning.Plan. INPUT=''


### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [19]:
var originalPlanResult = await kernel.RunAsync(originalPlan);

Console.WriteLine("Original Plan  results:\n");
Console.WriteLine(originalPlanResult.Result);



Original Plan  results:

If you and your partner are looking for a romantic activity, why not consider making a romantic dinner at home? Alternatively, you could take a walk in a scenic spot, have a picnic in the park, go to a drive-in movie, take a cooking class together, visit a local museum, go ice-skating, take a dance class, go to a local concert, or take a hot air balloon ride.


Now lets execute and print the new plan:

In [20]:
var newPlanResult = await kernel.RunAsync(newPlan);
Console.WriteLine("New  Plan  results:\n");
Console.WriteLine(newPlanResult.Result);


New  Plan  results:

1. Have a passionate repast in a park most fair.
2. To a romantic dinner go, and naught compare.
3. Take a saunter on the beach, and be most merry.
4. To a local winery go, and there be merry.
5. Watch a romantic movie, and be most content.
6. To a drive-in movie theater go, and be most blent.
7. Take a cooking class together, and be most merry.
8. Take a hot air balloon ride, and be most merry.
9. Go for a scenic hike, and be most content.
10. Stargaze in a field, and be most blent.
